Notebook for uploading PDF, extracting all Text and Pre-Processing using a 1B or 3B model

In [41]:
#!pip install PyPDF2
#!pip install rich ipywidgets

In [14]:
pdf_path = './2402.13116v3.pdf'
DEFAULT_MODEL = "meta-llama/Llama-3.2-1B-Instruct"
#DEFAULT_MODEL = "meta-llama/Llama-3.2-1B-Instruct" <- Don't think this would be necessary

In [31]:
from difflib import HtmlDiff
from IPython.display import HTML, display

In [49]:
# Import necessary libraries
import PyPDF2
from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer

from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

In [9]:
def validate_pdf(file_path: str) -> bool:
    if not os.path.exists(file_path):
        print(f"Error: File not found at path: {file_path}")
        return False
    if not file_path.lower().endswith('.pdf'):
        print("Error: File is not a PDF")
        return False
    return True

In [10]:
def extract_text_from_pdf(file_path: str, max_chars: int = 100000) -> Optional[str]:
    if not validate_pdf(file_path):
        return None
    
    try:
        with open(file_path, 'rb') as file:
            # Create PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)
            
            # Get total number of pages
            num_pages = len(pdf_reader.pages)
            print(f"Processing PDF with {num_pages} pages...")
            
            extracted_text = []
            total_chars = 0
            
            # Iterate through all pages
            for page_num in range(num_pages):
                # Extract text from page
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                
                # Check if adding this page's text would exceed the limit
                if total_chars + len(text) > max_chars:
                    # Only add text up to the limit
                    remaining_chars = max_chars - total_chars
                    extracted_text.append(text[:remaining_chars])
                    print(f"Reached {max_chars} character limit at page {page_num + 1}")
                    break
                
                extracted_text.append(text)
                total_chars += len(text)
                print(f"Processed page {page_num + 1}/{num_pages}")
            
            final_text = '\n'.join(extracted_text)
            print(f"\nExtraction complete! Total characters: {len(final_text)}")
            return final_text
            
    except PyPDF2.PdfReadError:
        print("Error: Invalid or corrupted PDF file")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None


In [11]:
# Get PDF metadata
def get_pdf_metadata(file_path: str) -> Optional[dict]:
    if not validate_pdf(file_path):
        return None
    
    try:
        with open(file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            metadata = {
                'num_pages': len(pdf_reader.pages),
                'metadata': pdf_reader.metadata
            }
            return metadata
    except Exception as e:
        print(f"Error extracting metadata: {str(e)}")
        return None

In [12]:
# Extract metadata first
print("Extracting metadata...")
metadata = get_pdf_metadata(pdf_path)
if metadata:
    print("\nPDF Metadata:")
    print(f"Number of pages: {metadata['num_pages']}")
    print("Document info:")
    for key, value in metadata['metadata'].items():
        print(f"{key}: {value}")

# Extract text
print("\nExtracting text...")
extracted_text = extract_text_from_pdf(pdf_path)

# Display first 500 characters of extracted text as preview
if extracted_text:
    print("\nPreview of extracted text (first 500 characters):")
    print("-" * 50)
    print(extracted_text[:500])
    print("-" * 50)
    print(f"\nTotal characters extracted: {len(extracted_text)}")

# Optional: Save the extracted text to a file
if extracted_text:
    output_file = 'extracted_text.txt'
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(extracted_text)
    print(f"\nExtracted text has been saved to {output_file}")

Extracting metadata...

PDF Metadata:
Number of pages: 44
Document info:
/Author: 
/CreationDate: D:20240311015030Z
/Creator: LaTeX with hyperref
/Keywords: 
/ModDate: D:20240311015030Z
/PTEX.Fullbanner: This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5
/Producer: pdfTeX-1.40.25
/Subject: 
/Title: 
/Trapped: /False

Extracting text...
Processing PDF with 44 pages...
Processed page 1/44
Processed page 2/44
Processed page 3/44
Processed page 4/44
Processed page 5/44
Processed page 6/44
Processed page 7/44
Processed page 8/44
Processed page 9/44
Processed page 10/44
Processed page 11/44
Processed page 12/44
Processed page 13/44
Processed page 14/44
Processed page 15/44
Processed page 16/44
Reached 100000 character limit at page 17

Extraction complete! Total characters: 100016

Preview of extracted text (first 500 characters):
--------------------------------------------------
1
A Survey on Knowledge Distillation of Large
Language Models
Xiaohan Xu1, M

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"

SYS_PROMPT = """
You are a world class text pre-processor, here is the raw data from a PDF, please parse and return it in a way that is crispy and usable to send to a podcast writer.

The raw data is messed up with new lines, Latex math and you will see fluff that we can remove completely. Basically take away any details that you think might be useless in a podcast author's transcript.

Remember, the podcast could be on any topic whatsoever so the issues listed above are not exhaustive

The goal is to use this in a podcast research transcript so a lot of the emails, citations, and things like that can be removed-please be smart with what you remove and be creative ok?

Remember DO NOT START SUMMARIZING THIS, YOU ARE ONLY CLEANING UP THE TEXT AND RETURNING AS IS

Be very smart and aggressive with removing details, you will get a running portion of the text and keep returning the processed text.

ALWAYS start your response directly with processed text and NO ACKNOWLEDGEMENTS about my questions ok?
Here is the text:
"""

In [22]:
accelerator = Accelerator()
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map=device,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)

In [50]:
def process_chunk(text_chunk, chunk_num):
    """Process a chunk of text and return both input and output for verification"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]
    
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            temperature=0.7,
            top_p=0.9,
            max_new_tokens=512
        )
    
    processed_text = tokenizer.decode(output[0], skip_special_tokens=True)[len(prompt):].strip()
    
    # Print chunk information for monitoring
    #print(f"\n{'='*40} Chunk {chunk_num} {'='*40}")
    print(f"INPUT TEXT:\n{text_chunk[:500]}...")  # Show first 500 chars of input
    print(f"\nPROCESSED TEXT:\n{processed_text[:500]}...")  # Show first 500 chars of output
    print(f"{'='*90}\n")
    
    return processed_text

In [51]:
INPUT_FILE = "./extracted_text.txt"  # Replace with your file path
CHUNK_SIZE = 1000  # Adjust chunk size if needed

# Read the file
with open(INPUT_FILE, 'r', encoding='utf-8') as file:
    text = file.read()

# Calculate number of chunks
num_chunks = (len(text) + CHUNK_SIZE - 1) // CHUNK_SIZE

# Cell 6: Process the file with ordered output
# Create output file name
output_file = f"clean_{os.path.basename(INPUT_FILE)}"

In [ ]:
processed_text = ""  # Initialize complete processed text
with open(output_file, 'w', encoding='utf-8') as out_file:
    for chunk_num in tqdm(range(num_chunks), desc="Processing chunks"):
        # Get chunk with overlap
        start_idx = chunk_num * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        
        chunk = text[start_idx:end_idx]
        
        # Process chunk and append to complete text
        processed_chunk = process_chunk(chunk, chunk_num)
        processed_text += processed_chunk + "\n"
        
        # Write chunk immediately to file
        out_file.write(processed_chunk + "\n")
        
        # Force flush the file to disk
        out_file.flush()

Processing chunks:   0%|          | 0/101 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
1
A Survey on Knowledge Distillation of Large
Language Models
Xiaohan Xu1, Ming Li2, Chongyang Tao3, Tao Shen4, Reynold Cheng1, Jinyang Li1,
Can Xu5, Dacheng Tao6, Tianyi Zhou2
1The University of Hong Kong2University of Maryland3Microsoft
4University of Technology Sydney5Peking University6The University of Sydney
{shawnxxh,chongyangtao,hishentao }@gmail.com {minglii,tianyi }@umd.edu
ckcheng@cs.hku.hk jl0725@connect.hku.hk
Abstract —In the era of Large Language Models (LLMs), Knowledge Distillati...

PROCESSED TEXT:
Tao Shen4, Reynold Cheng1, Jinyang Li1,
Can Xu5, Dacheng Tao6, Tianyi Zhou2
1The University of Hong Kong2University of Maryland3Microsoft
4University of Technology Sydney5Peking University6The University of Sydney
{shawnxxh,chongyangtao,hishentao }@gmail.com {minglii,tianyi }@umd.edu
ckcheng@cs.hku.hk...



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
ed knowledge to smaller models and its utility in model compression and self-
improvement. Our survey is meticulously structured around three foundational pillars: algorithm ,skill, and verticalization – providing
a comprehensive examination of KD mechanisms, the enhancement of specific cognitive abilities, and their practical implications
across diverse fields. Crucially, the survey navigates the intricate interplay between data augmentation (DA) and KD, illustrating how
DA emerges as a powerfu...

PROCESSED TEXT:
ulously structured around three foundational pillars: algorithm, skill, and verticalization – providing a comprehensive examination of knowledge distillation mechanisms, the enhancement of specific cognitive abilities, and their practical implications across diverse fields. Crucially, the survey navigates the intricate interplay between data augmentation (DA) and knowledge distillation, illustrating how DA emerges as a powerful paradigm within the knowledge disti

...

INPUT TEXT:
ch the LLM can
explore to generate novel knowledge, Parse( o, s)stands for
to parse the distillation example ( e.g., (x, y)) from the
teacher LLM’s output o(plus the input sin some cases),
andpTrepresents the teacher LLM with parameters θT.
Given the datasets D(kd)
Ibuilt for distillation, we then define
a learning objective as
L=X
ILI(D(kd)
I;θS), (2)
whereP
Idenotes there could be multiple tasks or skills
being distilled into one student model, LI(·;·)stands for a
specific learning objective, ...

PROCESSED TEXT:
on example ( e.g., (x, y)) from the
teacher LLM’s output o(plus the input sin some cases),
andpTrepresents the teacher LLM with parameters θT.
Given the datasets D(kd)
Ibuilt for distillation, we then define
a learning objective as
L=X
ILI(D(kd)
I;θS), (2)
where...



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
 LLMs (Eq.1), and ‘Distillation,’
centered on injecting this knowledge into student models
(Eq.2). We will elaborate on these two processes in the
subsequent sections.
3.1 Knowledge
This section focuses on the approaches to elicit knowledge
from teacher LLMs. According to the manners to acquire
knowledge, we divided them into Labeling ,Expansion ,DataCuration ,Feature ,Feedback , and Self-Knowledge . Figure 5
shows an illustration of these knowledge elicitation meth-
ods.
3.1.1 Labeling
Labeling...

PROCESSED TEXT:
aborate on these two processes in the
subsequent sections.
3.1 Knowledge
This section focuses on the approaches to elicit knowledge
from teacher LLMs. According to the manners to acquire
knowledge, we divided them into Labeling,Expansion,DataCuration,Feature,Feedback, and Self-Knowledge. Figure 5
shows an illustration of these knowledge elicitation methods....



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


INPUT TEXT:
lable through the
predefined Iandc. This process can be formulated as
follows:
D(lab)={x, y|x∼ X, y∼pT(y|I⊕c⊕x)}. (3)
Input xcould be sourced from existing NLP task
datasets, which serve as typical reservoirs for distillation
efforts. Numerous works have sought to harness the capa-
bilities of powerful LLMs as teachers for annotating dataset
samples across a range of tasks. For instance, efforts in
natural language understanding involve using LLMs to cat-
egorize text (Gilardi et al., 2023; Ding...

PROCESSED TEXT:
, y∼pT(y|I⊕c⊕x)}. (3)
Input xcould be sourced from existing NLP task
datasets, which serve as typical reservoirs for distillation
efforts. Numerous works have sought to harness the
capabilities of powerful LLMs as teachers for annotating dataset
samples across a range of tasks. For instance, efforts in
natural language understanding involve using LLMs to
categorize text (Gilardi et al., 2023; Ding et al., 2023a; He et al.,
2023a), while in natural language genera

In [ ]:
print(f"\nProcessing complete!")
print(f"Input file: {INPUT_FILE}")
print(f"Output file: {output_file}")
print(f"Total chunks processed: {num_chunks}")

# Preview the beginning and end of the complete processed text
print("\nPreview of final processed text:")
print("\nBEGINNING:")
print(processed_text[:1000])
print("\n...\n\nEND:")
print(processed_text[-1000:])